In [3]:
import numpy as np
from sympy  import Symbol,Add,Mul,Pow,Number,symbols
from sympy import  factor,srepr,sympify,expand,sqrt,simplify
import matplotlib.pyplot as plt

In [ ]:
def create_dictionary(m):
    m_bin= bin(m)[2:][::-1]
    m_dict={}
    for i,value in enumerate(m_bin):
        m_dict[i]= int(value)
    p_dict={}
    p_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        p_dict[i]= Symbol('p'+str(i))
    p_dict[int(np.ceil(len(m_bin)/2))-1]=1 # length half of m is being held.here need some generializtion to check out bit length of p and q optimum.

    q_dict={}
    q_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        q_dict[i]= Symbol('q'+str(i))
    q_dict[int(np.ceil(len(m_bin)/2))-1]=1

    n=  len(m_bin) + int(np.ceil(len(m_bin)/2)) - 1
    z_dict={}
    for i in range(n):
        for j in range(i+1):
            if i != j:
                if i >= len(m_bin) :
                    pass
                elif j ==0 :
                    pass
                else :
                    if i-j >= int(np.ceil(len(m_bin)/2))-1  :
                        z_dict[(j, i)] = 0
                        
                    else:
                        z_dict[(j, i)] =  Symbol('z'+str(j)+str(i))

        z_dict[(len(m_bin)-4, len (m_bin)-1)] =0
    return m_dict , p_dict ,q_dict , z_dict

In [68]:
def max_sum(clause):
    max=0
    if clause.func == Add :
        for t in clause.args :
            if   isinstance(t, Number) :
                max = max+ int(t)
            #elif isinstance(t, Number) and int(t) < 0:
            #    max -= int(t) 
                #print(max)
            
            if t.func == Mul :
                if  isinstance(t.args[0], Symbol)  :           ###changed  from  previous one
                    max= max+1

                if isinstance(t.args[0], Number) and t.args[0] > 0:
                    max = max + (int(t.args[0]))
                    
                if isinstance(t.args[0], Number) and t.args[0] < 0:
                    max = max + 0
                      
                     
                    
                if isinstance(t.args[0], Number) and  t.args[0] <0 :
                    
                    pass  
            if t.func == Symbol  :
                
                max=max+1
                print(max)
            #if isinstance(t ,Symbol):
             #   max=max+1
            
            '''if   isinstance(t, Number) :
                max = max+ int(t)
            #elif isinstance(t, Number) and int(t) < 0:
            #    max -= int(t) '''
              
                

            #else :
             #   max = max+1
      

    elif clause.func == Mul :
        if  isinstance(clause.args[0], Number) and  clause.args[0] >0 :
            max =max+ int(clause.args[0])
        if   isinstance(clause.args[0], Number) and  clause.args[0] <0 :
            max =max+ int(0)
            
        else :
            max=max+1

    elif clause.func == Symbol :
        max = 1
    elif isinstance(clause,Number):
        max=max+int(clause)
    print(max)

    return max

In [ ]:
def simplify_clause(clause, equation,i):
    
    clause=clause.subs(equation).expand()
    

    if clause.func == Add:
        for t in clause.args:
            if t .func == Mul and 'Pow' in  srepr(t):
                for s in t.args:
                    if s.func  == pow:
                        clause=clause.subs({s: s.args[0]})
            if t.func == pow :
                clause=clause -t +t.args[0]

            else :
                f_clause = factor(clause)
                if f_clause .func == Mul :
                    if isinstance(f_clause.args[0],Number):
                        clause = clause /f_clause.args[0]
    if i == 1:
        equation= rule_1(clause,equation)
        clause=clause.subs(equation).expand()
        num=max_sum(clause)
    if i == 2:
        equation= rule_2(clause,equation)
        clause=clause.subs(equation).expand()
        num=max_sum(clause)
    
    if i == 3:
        equation= rule_3(clause,equation)
        clause=clause.subs(equation).expand()
        num=max_sum(clause)
    if i == 4:
       equation= rule_4(clause,equation)
       clause=clause.subs(equation).expand()
       num=max_sum(clause)
       #print(clause,equation)

    if i == 5:
       equation= rule_5(clause,equation)
       clause=clause.subs(equation).expand()
       num=max_sum(clause)
    
     #  print(clause,equation)
    #equation=rule_3(clause,equation)
    #clause=clause.subs(equation).expand()
    

    return  clause ,equation

In [ ]:
def rule_1(clause,expression):
    negative =[]
  
    for t in clause.args :
        if t.func == Mul  and isinstance(t.args[0], Number)  and t.args[0]<0 :
            negative.append(t)
        

        
    if len(negative) >0 :
        for  t in negative:
            if   -t.args[0] >= max_sum(clause):
                var= t/t.args[0]
                expression[var] =0

    return expression

In [ ]:
def clause_final(clauses):
    expression={}

    for f in range(3):
   #if f==0 :
   #  for p in range(len(clauses)):
   #     clauses[p] = expand(sympify(clauses[p]**2))
         
   #      for i in range(2,5):
   #          clauses[p]=sympify(str(clauses[p]) . replace('**'+str(i),''))
   #else :
   #   pass
       for i in range(1,6):
      #clauses = create_clause(m_dict, p_dict, q_dict, z_dict)
      #for c in range(len(clauses)):
      #   print(clauses[c])       
          for p in range(len(clauses)):

            clauses[p] ,expression=(simplify_clause(clauses[p], expression,i))
         
      #   print(clauses[p])
    clauses1=[]
    for p in range(len(clauses)):
       clauses1.append(clauses[p]**2)
    clause_final=sympify(sum(clauses1).expand())
    clause_final=sympify(str(clause_final) . replace('**2',''))

    return clause_final            


In [ ]:
def retrieve_dict(clause_final):
    p_dict={}
    q_dict={}
    z_dict={}
    if clause_final.func==Add:
        for t in clause_final.args:
            if t.func==Mul  and type(t)!=int:
                for subterm in t.args :
                    if type(subterm)!= int :
                        for i in str(subterm).split('*'):
                            if 'p' in str(i):
                                for j in i.split('p'):
                                    #if len(p_dict[int(j)])!= 0:
                                        if j!='':
                                            p_dict[(j)] = i
  
                            elif 'q' in str(i):
                                for j in i.split('q'):
                                   #if len(q_dict[int(j)])!= 0:
                                        if j!='':
                                            q_dict[(j)] = i

                            elif 'z' in str(i):
                                for j in i.split('z'):
                                    #if len(z_dict[(j)])!= 0:
                                        if j!='':
                                            z_dict[(j)] = i
            if t.func !=Mul and type(t)!= int :
                if 'p' in str(t):
                    for j in str(t).split('p'):
                        if j!='':
                           #if len(p_dict[int(j)])!= 0:
                               p_dict[(j)] = str(t)
  
                elif 'q' in str(t):
                    for j in str(t).split('q'):
                        if j!='':
                           #if len(q_dict[int(j)])!= 0:
                                q_dict[(j)] = str(t)

                elif 'z' in str(t):
                    for j in str(t).split('z'):
                        if j!='':
                           #if len(q_dict[int(j)])!= 0:
                                z_dict[(j)] = str(t)
    
    num_variables= len(p_dict)+ len(q_dict) + len(z_dict)
    return  num_variables



In [ ]:
def rule_2(clause, expression):
    if clause.func == Add and len(clause.args) ==3 and len(list(clause.free_symbols))==2 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x'),list(clause.free_symbols)[1]: Symbol('y')})
        rule= Symbol('x') + Symbol('y') -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0] * list(clause.free_symbols)[1]] =0
            if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[list(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]
    return expression

def  rule_3(clause , expression):
    if clause.func == Add and len(clause.args) ==2 and len(list(clause.free_symbols))==2 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x'),list(clause.free_symbols)[1]: Symbol('y')})
        rule= Symbol('x') * Symbol('y') -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0]]=1
            expression[list(clause.free_symbols)[0]]=1
    return expression


def  rule_4(clause , expression):
    count=0
    if clause.func == Add and max_sum(clause) == len(list(clause.free_symbols)):
        for t in clause.args :
            if t.func != Mul :
                expression[t] =0
    
    else :
        for t in clause.args :
            if isinstance(t,Number):
                count= count+t
        if clause.func == Add and  len(list(clause.free_symbols)) ==- count :
            for t in clause.args :
                if t != count :
                    expression[t] = 1

    return expression  
def rule_5(clause, known_expressions):
    
    if clause.func == Add and len(clause.args) == 2:
        if len(clause.args[0].free_symbols) == 0:
            constant_a = clause.args[0]
            if clause.args[1].func == Mul:
                constant_b = clause.args[1].args[0]
                symbol = clause.args[1] / constant_b
                if isinstance(constant_a,Number) and isinstance(constant_b,Number):
                    if constant_a > 0 or constant_b < 0:
                    
                       known_expressions[symbol] = 1
    return known_expressions

In [ ]:
def num_variables_final(factor):
    m_dict,p_dict,q_dict,z_dict= create_dictionary(factor)
    clauses= create_clause(m_dict,p_dict,q_dict,z_dict)
    clause_final1=clause_final(clauses)
    num_variables=retrieve_dict(clause_final1)

    return num_variables

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def is_prime(factor):
    for n in range(2,int(factor**0.5)+1):
        if factor%n==0:
            return False
    return True

In [ ]:
l=[]
for i in range(20,400):
    if is_prime(i):
        l.append(i)
f=[]
for i in range(len(l)):
    for j in range(i+1,len(l)):
        if i!=j:
            f.append(l[i]* l[j])
print(f)

x=[]
y=[]
for i in range(len(f)):
    
    num_variables=num_variables_final(f[i])
    print(num_variables)
    x.append(f[i])
    y.append(num_variables)
    
a=plt.figure()
axes= a.add_axes([0.1,0.1,0.8,0.8])
axes.set_xlim([2,100000])
axes.set_ylim([0,140])
axes.scatter(x,y)
plt.show()       

In [ ]:
z=[]
for i in range (len(y)):
    z.append(2*y[i])
a=plt.figure()
axes= a.add_axes([0.1,0.1,0.8,0.8])
axes.set_xlim([2,160000])
axes.set_ylim([0,140])
axes.scatter(x,z)
plt.show()      

In [ ]:
def create_dictionary_1(m):
    m_bin = bin(m)[2:][::-1]
    m_dict={}
    for i,value in enumerate(m_bin):
        m_dict[i] =  value
    m_dict_1={}
    for i in range(1,len(m_bin)-3,2):
        m_dict_1[i] = 2*m_dict[i+1] + m_dict[i]
    m_dict_1[len(m_bin)-3]= m_dict[len(m_bin)-3]  + 2*  m_dict[len(m_bin)-2] + 4* m_dict[len(m_bin)-2]
    p_dict={}
    p_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        p_dict[i]= Symbol('p'+str(i))
    p_dict[int(np.ceil(len(m_bin)/2))-1]=1 # length half of m is being held.here need some generializtion to check out bit length of p and q optimum.

    q_dict={}
    q_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        q_dict[i]= Symbol('q'+str(i))
    q_dict[int(np.ceil(len(m_bin)/2))-1]=1

    n=  len(m_bin) + int(np.ceil(len(m_bin)/2)) - 1

    c_dict={}
    for i in  range(3,len(m_bin)-1):
        c_dict[i-2] = Symbol('c'+str(i-2))

    return m_dict_1,p_dict,q_dict,c_dict


In [ ]:
def create_clause_1(m,p,q,c):
    clauses=[]
    n=len(m)
    for i in range(1,n-2,2):
        clause=0
        for j in range(i,i+2):
            
            
        
            for k in range(j+1):
                clause+=2**(j-i)*q.get(k,0) * p.get(j-k,0)
            clause+= 2**(j-i)*c.get(i-2,0)


        for j in range(2 ,np.ceil(len(m)/2)-1):
            clause+= - 2**(j) * c.get(i+j-2) 



        
        clause  += - m.get(i,0)

         
        if type(clause) == int:
            clause = sympify(clause)

        if clause ==0 :
            clause = sympify(clause)


        clauses.append(clause)


    return  clauses

             

In [4]:
def create_dictionary_robust(m):
    m_bin=bin(m)[2:][::-1]
    m_dict={}
    for i,j in enumerate(m_bin):
        m_dict[i] =int (j)

    p_dict={}
    p_dict[0]=1
    
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        p_dict[i]= Symbol('p'+str(i))
    p_dict[int(np.ceil(len(m_bin)/2))-1]=1 # length half of m is being held.here need some generializtion to check out bit length of p and q optimum.

    q_dict={}
    q_dict[0]=1
    for i in range(1,int(np.ceil(len(m_bin)/2))):
        q_dict[i]= Symbol('q'+str(i))
    q_dict[int(np.ceil(len(m_bin)/2))-1]=1

    n=  len(m_bin) + int(np.ceil(len(m_bin)/2)) - 1
    z_dict1={}
    z_dict2={}

    for i in range(2,len(m_bin)):
        for j in range(i-int(np.floor(np.log2(i))),i):
            if i!= j:
                z_dict1[(j, i)] =  Symbol('z'+str(j)+str(i))

    for i in range(1,len(m_bin)-1):
        for j in range(i,i+len(bin(i)[2:][::-1])+1):
            if i!= j  and j <= len(m_bin):
                z_dict2[(i, j)] =  Symbol('z'+str(i)+str(j))




    return m_dict , p_dict ,q_dict , z_dict2
                



    


In [ ]:
create_dictionary_robust(56153)

In [5]:
def create_clause1(m,p,q,z):
    clauses=[]
    clauses1=[]
    z_zero={}
    n=len(m)+int(np.ceil(len(m)/2)) -1
    for i in range(n):
        clause=0
        for j in range(i+1):
            clause+=q.get(j,0) * p.get(i-j,0)
        #clause  += - m.get(i,0)
        for j in range(i+1):
            clause += z.get((j, i), 0)
        
        if type(clause) == int:
            clause = sympify(clause)
        if clause != 0:
            max_sum1=max_sum(clause)
            if max_sum1 !=0  :
                max_carry = int(np.floor(np.log2(max_sum1)))
            else : 
                max_carry  = 0
        for j in range(len(z)):
            if j-i > max_carry:
                if z.get((i-1, j), 0) != 0:
                    z[(i-1, j)] = 0
                    z_zero['z'+str(i-1)+str(j)] = 0 
        clause  += - m.get(i,0)
        for j in range(i+1, i+5):
            if j-i <=  max_carry +1 :
               clause += - 2**(j-i) * z.get((i, j), 0) 
        
        
        if clause ==0 :
            clause = sympify(clause)
        clauses.append(clause)

    #for clause in clauses:
     #   for keys in z_zero:
     #       clause=str(clause).replace(z_zero[keys],'0')
     #   clauses1.append(clause)
            

    return clauses

In [ ]:
m,p,q,z=create_dictionary_robust(143 )
p=create_clause1(m,p,q,z)
p

In [6]:
def rule_21(clause, expression):
    if clause.func == Add and len(clause.args) ==3 and len(list(clause.free_symbols))==2 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x'),list(clause.free_symbols)[1]: Symbol('y')})
        rule= Symbol('x') + Symbol('y') -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0] * list(clause.free_symbols)[1]] =0
            expression[list(clause.free_symbols)[0] + list(clause.free_symbols)[1]] =1
            '''if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[lisit(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]'''
    if (len(expression)!= 0):
        print(('rule21applied'))
    print(expression)
    return expression

In [7]:
def rule_31(clause, expression):
    if clause.func == Add and len(clause.args) ==2 and len(list(clause.free_symbols))==1 :
        sub_clause=clause.subs({list(clause.free_symbols)[0]: Symbol('x')})
        rule= Symbol('x')  -1
        if sub_clause -rule ==0 :
            expression[list(clause.free_symbols)[0]] =1
            '''if 'q' in str(list(clause.free_symbols)[0]):
                expression[list(clause.free_symbols)[0]]= 1 - list(clause.free_symbols)[1]
            else :
                expression[list(clause.free_symbols)[1]]= 1 - list(clause.free_symbols)[0]'''
    if (len(expression)!= 0):
        print(('rule31applied'))
    print(expression)
    return expression

In [8]:
def rule_11(clause,expression):
    negative =[]
  
    for t in clause.args :
        if t.func == Mul  and isinstance(t.args[0], Number)  and t.args[0]<0 :
            negative.append(t)
        

        
    if len(negative) >0 :
        for  t in negative:
            if   -t.args[0] > max_sum(clause):
                var= t/t.args[0]
                expression[var] =0
    if (len(expression)!= 0):
        print(('rule11applied'))
    print(expression)
    #print (max_sum(clause))

    return expression

In [9]:
def rule_51(clause, known_expressions):
    
    if clause.func == Add and len(clause.args) == 2:
        if len(clause.args[0].free_symbols) == 0:
            constant_a = clause.args[0]
            if clause.args[1].func == Mul:
                constant_b = clause.args[1].args[0]
                symbol = clause.args[1] / constant_b
                if isinstance(constant_a,Number) and isinstance(constant_b,Number):
                    if constant_a > 0 or constant_b < 0:
                    
                       known_expressions[symbol] = 1
    return known_expressions

In [28]:
def  rule_41(clause , expression):
    count=0
    if clause.func == Add and max_sum(clause) == len(list(clause.free_symbols)):
        for t in clause.args :
            if t.func != Mul :
                expression[t] =0
    
    else :
        for t in clause.args :
            if isinstance(t,Number):
                count= count+t
        if clause.func == Add and  len(list(clause.free_symbols)) ==- count :
            for t in clause.args :
                if t != count :
                    expression[t] = 1

In [ ]:
expression= {}
for clauses in p:
    rule_11(clauses,expression)
    

In [ ]:
renew_clause=[]
for clauses in p:
    renew_clause.append(clauses.subs(expression).expand())
renew_clause

In [ ]:
renew_clause1=[]
for clauses in renew_clause:
    rule_21(clauses,expression)

for clauses in p:
    renew_clause1.append(clauses.subs(expression).expand())
renew_clause1


In [ ]:
renew_clause2=[]
for clauses in renew_clause:
    rule_11(clauses,expression)

for clauses in p:
    renew_clause2.append(clauses.subs(expression).expand())
renew_clause2


In [ ]:
renew_clause3=[]
for clauses in renew_clause:
    rule_11(clauses,expression)

for clauses in p:
    renew_clause3.append(clauses.subs(expression).expand())
renew_clause3

In [ ]:
renew_clause4=[]
for clauses in renew_clause:
    rule_31(clauses,expression)

for clauses in p:
    renew_clause4.append(clauses.subs(expression).expand())
renew_clause4

In [ ]:
renew_clause5=[]
for clauses in renew_clause4:
    rule_31(clauses,expression)

for clauses in renew_clause4:
    renew_clause5.append(clauses.subs(expression).expand())
renew_clause5

In [ ]:
renew_clause6=[]
for clauses in renew_clause5:
    rule_21(clauses,expression)

for clauses in renew_clause5:
    renew_clause6.append(clauses.subs(expression).expand())
renew_clause6

In [ ]:
renew_clause7=[]
for clauses in renew_clause6:
    rule_31(clauses,expression)

for clauses in renew_clause6:
    renew_clause7.append(clauses.subs(expression).expand())
renew_clause7

In [ ]:
renew_clause8=[]
for clauses in renew_clause7:
    rule_31(clauses,expression)

for clauses in renew_clause7:
    renew_clause8.append(clauses.subs(expression).expand())
renew_clause8

In [ ]:
for key in expression:
    if expression[key]!=0  :
       renew_clause8.append(key - expression[key])
renew_clause8

In [13]:
m,p,q,z = create_dictionary_robust(3599)
p2=create_clause1(m,p,q,z)
p2


1
2
4
5
7
8
7
7
6
5
4
3
2


[0,
 p1 + q1 - 2*z12 - 1,
 p1*q1 + p2 + q2 + z12 - 2*z23 - 4*z24 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z24 + z34 - 2*z45 - 4*z46 - 8*z47,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p1 + p2*q4 + p3*q3 + p4*q2 + q1 + z46 + z56 - 2*z67 - 4*z68 - 8*z69,
 p2 + p3*q4 + p4*q3 + q2 + z47 + z57 + z67 - 8*z710 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 8*z811 - 2*z89,
 p4 + q4 + z69 + z79 + z89 - 2*z910 - 4*z911 - 8*z912 - 1,
 -2*z1011 - 4*z1012 + z710 + z810 + z910,
 z1011 + z811 + z911 - 1,
 z1012 + z912,
 0,
 0,
 0,
 0]

In [19]:
expression_3599={}
for clauses in p2:
    
    rule_11(clauses,expression_3599)


{}
1
rule11applied
{z12: 0}
3
3
rule11applied
{z12: 0, z24: 0}
4
4
rule11applied
{z12: 0, z24: 0}
7
7
7
rule11applied
{z12: 0, z24: 0, z47: 0}
8
8
8
rule11applied
{z12: 0, z24: 0, z47: 0}
7
7
7
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0}
7
7
7
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0}
6
6
6
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0}
4
4
4
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0}
3
3
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0}
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0}
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0}
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0}
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0}
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0}
rule11applied
{z12: 0, z24: 0, z47: 0,

In [21]:
renew_clause10=[]
for clauses in  renew_clause11:
    rule_21(clauses,expression_3599)
for clauses in  renew_clause11 :
    renew_clause10.append(clauses.subs(expression_3599).expand())
renew_clause10

rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z91

[0,
 0,
 p2 + q2 - 2*z23 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p2 + p3*q4 + p4*q3 + q2 + z57 + z67 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 4*z911 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [20]:
renew_clause11=[]
for clauses in p2:
    renew_clause11.append(clauses.subs(expression_3599).expand())
renew_clause11

[0,
 p1 + q1 - 1,
 p1*q1 + p2 + q2 - 2*z23 - 1,
 p1*q2 + p2*q1 + p3 + q3 + z23 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p1 + p2*q4 + p3*q3 + p4*q2 + q1 + z46 + z56 - 2*z67 - 4*z68,
 p2 + p3*q4 + p4*q3 + q2 + z57 + z67 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 4*z911 - 1,
 -2*z1011 + z810 + z910,
 z1011 + z911 - 1,
 0,
 0,
 0,
 0,
 0]

In [22]:

for clauses in renew_clause10:
    rule_11(clauses,expression_3599)

rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1}
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1}
1
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0}
4
4
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0}
6
6
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0}
8
8
8
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0}
6
6
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0,

In [24]:
renew_clause12=[]
for clauses in renew_clause10:
    renew_clause12.append(clauses.subs(expression_3599).expand())
renew_clause12

[0,
 0,
 p2 + q2 - 1,
 p1*q2 + p2*q1 + p3 + q3 - 2*z34 - 4*z35 - 1,
 p1*q3 + p2*q2 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z35 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p2 + p3*q4 + p4*q3 + q2 + z57 + z67 - 2*z78 - 4*z79,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [26]:
renew_clauses13=[]
for clauses in renew_clause12 :
    rule_21(clauses,expression_3599)

for clauses in renew_clause12 :
    renew_clauses13.append(clauses.subs(expression_3599).expand())
renew_clauses13

rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0, p2*q2: 0, p2 + q2: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0, p2*q2: 0, p2 + q2: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0, p2*q2: 0, p2 + q2: 1}
rule21applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0

[0,
 0,
 0,
 p1*q2 + p2*q1 + p3 + q3 - 2*z34 - 1,
 p1*q3 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z45 - 2*z56 - 4*z57 - 8*z58 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p3*q4 + p4*q3 + z57 + z67 - 2*z78 - 4*z79 + 1,
 p3 + p4*q4 + q3 + z58 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [27]:
renew_clauses14=[]
for clauses in renew_clauses13 :
    rule_11(clauses,expression_3599)

for clauses in renew_clauses13 :
    renew_clauses14.append(clauses.subs(expression_3599).expand())
renew_clauses14


rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0, p2*q2: 0, p2 + q2: 1}
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0, p2*q2: 0, p2 + q2: 1}
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0, p2*q2: 0, p2 + q2: 1}
3
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0, p2*q2: 0, p2 + q2: 1}
5
5
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0, z912: 0, z1012: 0, p1*q1: 0, p1 + q1: 1, z1011*z911: 0, z1011 + z911: 1, z23: 0, z911: 0, z35: 0, p2*q2: 0, p2 + q2: 1}
7
7
7
rule11applied
{z12: 0, z24: 0, z47: 0, z69: 0, z710: 0, z811: 0

[0,
 0,
 0,
 p1*q2 + p2*q1 + p3 + q3 - 2*z34 - 1,
 p1*q3 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z45 - 2*z56 - 4*z57 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p3*q4 + p4*q3 + z57 + z67 - 2*z78 - 4*z79 + 1,
 p3 + p4*q4 + q3 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [29]:
renew_clauses15=[]
for clauses in renew_clauses14 :
    rule_41(clauses,expression_3599)

for clauses in renew_clauses14 :
    renew_clauses15.append(clauses.subs(expression_3599).expand())
renew_clauses15

3
5
7
6
5
5
3
2


[0,
 0,
 0,
 p1*q2 + p2*q1 + p3 + q3 - 2*z34 - 1,
 p1*q3 + p3*q1 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p1*q4 + p2*q3 + p3*q2 + p4*q1 + z45 - 2*z56 - 4*z57 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p3*q4 + p4*q3 + z57 + z67 - 2*z78 - 4*z79 + 1,
 p3 + p4*q4 + q3 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [33]:
renew_clauses16=[]
'''for clauses in renew_clauses15 :
    rule_41(clauses,expression_3599)'''

for clauses in renew_clauses15 :
    renew_clauses16.append(clauses.subs(expression_3599).expand())
renew_clauses16

[0,
 0,
 0,
 p2 + p3 + q3 - 2*z34 - 1,
 p3 + p4 + q4 + z34 - 2*z45 - 4*z46,
 p2*q3 + p3*q2 + p4 + z45 - 2*z56 - 4*z57 + 2,
 p2*q4 + p3*q3 + p4*q2 + z46 + z56 - 2*z67 - 4*z68 + 1,
 p3*q4 + p4*q3 + z57 + z67 - 2*z78 - 4*z79 + 1,
 p3 + p4*q4 + q3 + z68 + z78 - 4*z810 - 2*z89,
 p4 + q4 + z79 + z89 - 2*z910 - 1,
 -2*z1011 + z810 + z910,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
renew_clauses17=[]
for clauses in renew_clauses16 :
    rule_21(clauses,expression_2)

for clauses in renew_clauses16 :
    renew_clauses17.append(clauses.subs(expression_2).expand())
renew_clauses17

In [ ]:
renew_clauses18=[]
for clauses in renew_clauses17 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses17 :
    renew_clauses18.append(clauses.subs(expression_2).expand())
renew_clauses18

In [ ]:
renew_clauses19=[]
for clauses in renew_clauses18 :
    rule_41(clauses,expression_2)

for clauses in renew_clauses18 :
    renew_clauses19.append(clauses.subs(expression_2).expand())
renew_clauses19

In [ ]:
renew_clauses20=[]
for clauses in renew_clauses19 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses19 :
    renew_clauses20.append(clauses.subs(expression_2).expand())
renew_clauses20

In [ ]:
renew_clauses21=[]
for clauses in renew_clauses20 :
    rule_41(clauses,expression_2)

for clauses in renew_clauses20 :
    renew_clauses21.append(clauses.subs(expression_2).expand())
renew_clauses21

In [ ]:
renew_clauses22=[]
for clauses in renew_clauses21 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses21 :
    renew_clauses22.append(clauses.subs(expression_2).expand())
renew_clauses22

In [ ]:
renew_clauses23=[]
for clauses in renew_clauses22 :
    rule_31(clauses,expression_2)

for clauses in renew_clauses22 :
    renew_clauses23.append(clauses.subs(expression_2).expand())
renew_clauses23

In [ ]:
renew_clauses24=[]
for clauses in renew_clauses23 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses23:
    renew_clauses24.append(clauses.subs(expression_2).expand())
renew_clauses24

In [ ]:
renew_clauses25=[]
for clauses in renew_clauses24 :
    rule_31(clauses,expression_2)

for clauses in renew_clauses24:
    renew_clauses25.append(clauses.subs(expression_2).expand())
renew_clauses25

In [ ]:
renew_clauses26=[]
for clauses in renew_clauses25 :
    rule_11(clauses,expression_2)

for clauses in renew_clauses25:
    renew_clauses26.append(clauses.subs(expression_2).expand())
renew_clauses26

In [ ]:
renew_clauses27=[]
for clauses in renew_clauses26 :
    rule_31(clauses,expression_2)

for clauses in renew_clauses26:
    renew_clauses27.append(clauses.subs(expression_2).expand())
renew_clauses27

In [ ]:
for key in expression_2:
    if expression_2[key]!=0  :
       renew_clauses27.append(key - expression_2[key])
renew_clauses27

In [ ]:
m,p,q,z = create_dictionary_robust(56153)
p1=create_clause1(m,p,q,z)
p1

In [ ]:
expression_33 ={}
renew_clauses31=[]
for clauses in p1 :
    rule_11(clauses,expression_33)

for clauses in p1:
    renew_clauses31.append(clauses.subs(expression_33).expand())
renew_clauses31

In [ ]:
expression_4={}
expression_4[Symbol('p1')]=0
expression_4[Symbol('p2')]=0
expression_4[Symbol('q1')]=0
expression_4[Symbol('q2')]=0
expression_4[Symbol('z12')]=0
expression_4[Symbol('z23')]=0
expression_4[Symbol('z34')]=0
expression_4[Symbol('z24')]=0
#expression_4[Symbol('q6')]=0


expression_4

In [ ]:
renew_clauses32=[]
for clauses in renew_clauses31 :
    rule_11(clauses,expression_4)

for clauses in  renew_clauses31:
    renew_clauses32.append(clauses.subs(expression_4).expand())
renew_clauses32

In [ ]:
renew_clauses33=[]
for clauses in renew_clauses32 :
    rule_11(clauses,expression_4)

for clauses in  renew_clauses32:
    renew_clauses33.append(clauses.subs(expression_4).expand())
renew_clauses33

In [ ]:
renew_clauses34=[]
for clauses in renew_clauses33 :
    rule_11(clauses,expression_4)

for clauses in  renew_clauses33:
    renew_clauses34.append(clauses.subs(expression_4).expand())
renew_clauses34

In [ ]:
expression_5 = {}
expression_5[Symbol('p5')] = 1
expression_5[Symbol('q5')] = 1 
expression_5[Symbol('p6')] = 1
expression_5[Symbol('q6')] = 1 
expression_5[Symbol('z56')] = 1 
expression_5[Symbol('z67')] = 1
expression_5[Symbol('z79')] = 1 



renew_clauses35=[]
for clauses in renew_clauses34 :
    rule_11(clauses,expression_5)

for clauses in  renew_clauses34:
    renew_clauses35.append(clauses.subs(expression_5).expand())
renew_clauses35

In [ ]:
expression_6={}
expression_6[Symbol('p1')]=1
expression_6[Symbol('q1')] =1
expression_6[Symbol('z710')] =1
expression_6[Symbol('z810')] =1
expression_6[Symbol('z910')] =1
renew_clauses34=[]
for clauses in renew_clauses33 :
    rule_11(clauses,expression_6)

for clauses in  renew_clauses33:
    renew_clauses34.append(clauses.subs(expression_6).expand())
renew_clauses34


In [105]:
_ih[-10:]

['expression_4036 = {}\nrenew_clauses2357=[]\nfor clauses in renew_clauses2356 :\n    rule_71(clauses,expression_4036)\nfor clauses in  renew_clauses2356 :\n    renew_clauses2357.append(clauses.subs(expression_4036))\n\nrenew_clauses2357\n    ',
 'def rule_71(clause,expression):\n    count=0\n    constant =0\n    negative=[]\n    if clause.func==Add  :\n        for t in clause.args:\n            if isinstance(t, Number) and int(t) > 0:\n                constant=t\n            if  isinstance(t,Symbol):\n                count = count +1\n            if t.func==Mul and t.args[0] < 0:\n                negative.append(- t.args[0])\n\n\n\n    if len(negative) > 0 and constant+count == max(negative) and constant > 0:\n        for t in clause.args:\n            if t.func == Mul  and isinstance(t.args[0] , Number) :\n                var=t/t.args[0]\n                expression[var] = 1\n                for i in var.args:\n                    expression[i] =1\n            else:\n                i